In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ball = pd.read_csv("/Users/annadaugaard/Desktop/VFF/explore/ball_tracking_VFF.csv")

In [ ]:
ball = ball.dropna()

In [ ]:


# Calculate differences to compute speed
ball['dx'] = ball['ball_x'].diff()
ball['dy'] = ball['ball_y'].diff()
ball['dt'] = ball['time'].diff()

# Calculate speed (Euclidean distance per time difference)
ball['speed'] = np.sqrt(ball['dx']**2 + ball['dy']**2) / ball['dt']

ball['acceleration'] = ball["speed"].diff() / ball['time'].diff()
ball = ball[ball['speed'] <= 36]
# Drop intermediate calculation columns from the cleaned tracking data
ball.drop(columns=['dx', 'dy', 'dt'], inplace=True)


window_size = 5
# Create a new column for the moving average smoothed acceleration
ball['smoothed_acceleration'] = ball['acceleration'].rolling(window=window_size, center=True).mean()
ball['smoothed_acceleration_observed'] = [1 if abs(x) >= 10 else 0 for x in ball['acceleration']]


In [ ]:

# Plot for comparison
plt.figure(figsize=(10,6))
plt.plot(ball['time'][0:100], ball['acceleration'][0:100], label='Original Acceleration', alpha=0.6)
plt.plot(ball['time'][0:100], ball['smoothed_acceleration'][0:100], label='Smoothed (Moving Average)', linewidth=2)
plt.axhline(y=10, color='red', linestyle='--', linewidth=1.5) 
plt.axhline(y=-10, color='red', linestyle='--', linewidth=1.5)
plt.xlabel("Time")
plt.ylabel("Acceleration")
plt.legend()
plt.title("Acceleration Smoothing with Moving Average")
plt.show()

In [ ]:
# Create a density plot on the column 'my_column'
plt.figure(figsize=(10, 6))
sns.kdeplot(data=ball, x='smoothed_acceleration_observed', fill=True, color='skyblue')
plt.title('Density Plot of my_column')
plt.xlabel('my_column')
plt.ylabel('Density')
plt.show()

In [ ]:
viborg_players = pd.read_csv("/Users/annadaugaard/Desktop/VFF/explore/viborg_players_tracking.csv")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Example DataFrame (replace this with your actual DataFrame)
data = {
    'player_id': ['037bde83-261d-4062-8ea8-a18a3e7f7ad0', '20369dd6-c959-427c-80ce-4c9b4d8bf888',
                  '261c7d62-d32e-44c8-a728-5b62df237386', '2a5f69ac-5fbb-48f2-9f0b-f34f93bcb8c9',
                  '2cccce55-d61b-4e71-8b42-29e11b1fb204', '35fccdd1-0ce2-44b7-81d6-02c12f74e1bd',
                  '3fe4590a-9b62-4862-b173-96a2b254693e', '45acfe4e-4812-4854-a8bb-6326c972d185',
                  '49b5e848-248d-4c03-96a9-c2a7d391f8b5', '4ade4034-c4d7-4c16-8984-0cef41d37f35',
                  '587ba55f-2607-4e5d-8b6f-8b94c119b81d', '61414b28-9922-4593-aef6-763b7fecc38b',
                  '72cc31c2-1e46-4249-900f-a4158dbe438d', '7b1d8a9a-f81b-4b5c-b9bb-4d9c97c745ad',
                  '8dad1822-e63d-4fde-8ebf-77740c65b478', '9f93778d-7f94-43e3-b02e-7ae424074c9e',
                  'a5f06ef3-cfe8-4d24-b6fd-ccc350ed844c', 'b82b47f6-922d-4cf7-a84e-b4309d7431ae',
                  'd498635e-afde-4024-b06b-4840308ef8a7', 'd6fdf0d3-e202-41ac-89d8-d6e5094fc449',
                  'dbc303d2-8d8b-41f9-b2df-5f37b19480b0', 'f85a2d16-66b4-4257-a907-f97486da6a83'],
    'unique_id': ['viborg'] * 22,
    'period': [1] * 22,
    'time': [0.0] * 22,
    # These numbers correspond to jersey numbers (player_num)
    'player_num': [12, 14, 11, 8, 28, 23, 18, 24, 10, 6, 16, 17, 10, 20, 11, 13, 4, 19, 16, 2, 1, 13],
    'x': [52.77, 36.87, 52.89, 54.96, 53.07, 57.54, 67.18, 73.17, 54.40, 36.80, 91.50, 44.18,
          53.61, 36.58, 49.04, 35.33, 36.03, 53.44, 42.05, 71.15, 7.73, 60.39],
    'y': [33.78, 45.35, 21.82, 45.76, 5.80, 12.03, 56.73, 36.32, 10.50, 15.22, 33.69, 30.64,
          47.35, 15.31, 11.37, 31.65, 25.80, 25.20, 21.88, 20.55, 33.85, 26.58],
    'z': [0.0] * 22,
    'spd': [0.0] * 22,
    'Team':['home',
 'away',
 'home',
 'home',
 'home',
 'home',
 'home',
 'home',
 'home',
 'away',
 'home',
 'away',
 'away',
 'away',
 'away',
 'away',
 'away',
 'away',
 'away',
 'home',
 'away',
 'home']
}

df = pd.DataFrame(data)

# Your dictionary for home team numbers
#teams_dict = {"home": [16, 24, 18, 8, 12, 13, 11, 10, 23, 28, 2]}

# Assign team based on player_num membership in teams_dict["home"]
# If player_num is in the list, assign "home", else assign "away"
#df['team'] = df['player_num'].apply(lambda x: 'home' if x in teams_dict["home"] else 'away')

# Identify duplicated player numbers (those that appear more than once)
duplicates = df[df.duplicated(subset='player_num', keep=False)]

# Plot only the duplicates so you can visually inspect them
plt.figure(figsize=(8, 6))
plt.scatter(df['x'], df['y'], color='blue', label='Duplicate Player Positions')

# Annotate each duplicate with its player_num and player_id (or any other identifier)
for idx, row in df.iterrows():
    label_text = f"{row['player_num']}"  # show player_num and first 8 chars of id
    if row["Team"] == "home":
          color = "green"
    else: 
          color = "red"
    plt.annotate(label_text, (row['x'], row['y']),
                 textcoords="offset points", xytext=(5, 5),
                 fontsize=9, color=color)

plt.xlim(0, 106)  # Set x-axis limits
plt.ylim(0, 68)
# Draw the reference lines for the new origin
plt.axhline(34, color='black', linewidth=0.5)
plt.axvline(53, color='black', linewidth=0.5)

# Plot a red cross at the new origin
plt.scatter(53, 34, color='red', marker='x', s=100, label='New Origin (0,0)')

plt.title('First loc positions for all players')
plt.grid(True)
plt.legend()
plt.show()

# At this point, visually inspect the plot.
# You can then decide (manually or programmatically) which duplicate should be assigned which team.
# For example, you might update the 'team' column for a specific duplicate as follows:
# df.loc[df['player_id'] == 'd6fdf0d3-e202-41ac-89d8-d6e5094fc449', 'team'] = 'home'
# (Replace the condition with your selection criteria.)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Example DataFrame (replace this with your actual DataFrame)
data = {
    'player_id': ['037bde83-261d-4062-8ea8-a18a3e7f7ad0', '20369dd6-c959-427c-80ce-4c9b4d8bf888',
                  '261c7d62-d32e-44c8-a728-5b62df237386', '2a5f69ac-5fbb-48f2-9f0b-f34f93bcb8c9',
                  '2cccce55-d61b-4e71-8b42-29e11b1fb204', '35fccdd1-0ce2-44b7-81d6-02c12f74e1bd',
                  '3fe4590a-9b62-4862-b173-96a2b254693e', '45acfe4e-4812-4854-a8bb-6326c972d185',
                  '49b5e848-248d-4c03-96a9-c2a7d391f8b5', '4ade4034-c4d7-4c16-8984-0cef41d37f35',
                  '587ba55f-2607-4e5d-8b6f-8b94c119b81d', '61414b28-9922-4593-aef6-763b7fecc38b',
                  '72cc31c2-1e46-4249-900f-a4158dbe438d', '7b1d8a9a-f81b-4b5c-b9bb-4d9c97c745ad',
                  '8dad1822-e63d-4fde-8ebf-77740c65b478', '9f93778d-7f94-43e3-b02e-7ae424074c9e',
                  'a5f06ef3-cfe8-4d24-b6fd-ccc350ed844c', 'b82b47f6-922d-4cf7-a84e-b4309d7431ae',
                  'd498635e-afde-4024-b06b-4840308ef8a7', 'd6fdf0d3-e202-41ac-89d8-d6e5094fc449',
                  'dbc303d2-8d8b-41f9-b2df-5f37b19480b0', 'f85a2d16-66b4-4257-a907-f97486da6a83'],
    'unique_id': ['viborg'] * 22,
    'period': [1] * 22,
    'time': [0.0] * 22,
    'player_num': [12, 14, 11, 8, 28, 23, 18, 24, 10, 6, 16, 17, 10, 20, 11, 13, 4, 19, 16, 2, 1, 13],
    'x': [52.77, 36.87, 52.89, 54.96, 53.07, 57.54, 67.18, 73.17, 54.40, 36.80, 91.50, 44.18,
          53.61, 36.58, 49.04, 35.33, 36.03, 53.44, 42.05, 71.15, 7.73, 60.39],
    'y': [33.78, 45.35, 21.82, 45.76, 5.80, 12.03, 56.73, 36.32, 10.50, 15.22, 33.69, 30.64,
          47.35, 15.31, 11.37, 31.65, 25.80, 25.20, 21.88, 20.55, 33.85, 26.58],
    'z': [0.0] * 22,
    'spd': [0.0] * 22,
    'Team': ['home', 'away', 'home', 'home', 'home', 'home', 'home', 'home', 'home', 'away',
             'home', 'away', 'away', 'away', 'away', 'away', 'away', 'away', 'away', 'home', 'away', 'home']
}

df = pd.DataFrame(data)

# Identify duplicated player numbers
duplicates = df[df.duplicated(subset='player_num', keep=False)]

# Define function to draw football field
def draw_pitch(ax):
    # Outer boundary
    ax.set_xlim(0, 106)
    ax.set_ylim(0, 68)
    ax.add_patch(patches.Rectangle((0, 0), 106, 68, linewidth=2, edgecolor='black', facecolor='none'))
    
    # Center circle
    center_circle = patches.Circle((53, 34), 9.15, color='black', fill=False, linewidth=1.5)
    ax.add_patch(center_circle)
    
    # Halfway line
    ax.plot([53, 53], [0, 68], color="black", linewidth=1.5)
    
    # Goals (rectangles at both ends)
    goal_left = patches.Rectangle((-2, 24), 2, 20, linewidth=1.5, edgecolor="black", facecolor="none")
    goal_right = patches.Rectangle((106, 24), 2, 20, linewidth=1.5, edgecolor="black", facecolor="none")
    
    ax.add_patch(goal_left)
    ax.add_patch(goal_right)
    
    # Penalty areas
    penalty_left = patches.Rectangle((0, 13), 16.5, 42, linewidth=1.5, edgecolor="black", facecolor="none")
    penalty_right = patches.Rectangle((89.5, 13), 16.5, 42, linewidth=1.5, edgecolor="black", facecolor="none")
    
    ax.add_patch(penalty_left)
    ax.add_patch(penalty_right)
    
    # Small goal areas
    small_box_left = patches.Rectangle((0, 26), 5.5, 16, linewidth=1.5, edgecolor="black", facecolor="none")
    small_box_right = patches.Rectangle((100.5, 26), 5.5, 16, linewidth=1.5, edgecolor="black", facecolor="none")
    
    ax.add_patch(small_box_left)
    ax.add_patch(small_box_right)
    
    # Penalty spots
    ax.scatter(11, 34, color="black", s=20)
    ax.scatter(95, 34, color="black", s=20)

# Create figure
fig, ax = plt.subplots(figsize=(10, 6))
draw_pitch(ax)

# Scatter player positions
for idx, row in df.iterrows():
    color = "blue" if row["Team"] == "home" else "red"
    ax.scatter(row["x"], row["y"], color=color, label=f"Player {row['player_num']}" if row['player_num'] not in df['player_num'][:idx].values else "")

    # Annotate players
    ax.annotate(str(row['player_num']), (row['x'], row['y']), textcoords="offset points",
                xytext=(5, 5), fontsize=9, color=color)

# Mark center spot
ax.scatter(53, 34, color='red', marker='x', s=100, label='New Origin (0,0)')

# Remove duplicate legends
handles, labels = ax.get_legend_handles_labels()
unique_labels = dict(zip(labels, handles))
#ax.legend(unique_labels.values(), unique_labels.keys(), loc='upper right')

# Title
ax.set_title("Football Field with Player Positions (blue home team, red away team)")

# Show plot
plt.show()

#byqbib-Mufjab-2xibqu


In [ ]:
# Your dictionary defining home team ids
away = {"away":["72cc31c2","9f93778d","d498635e","8dad1822"]}
teams_dict = {"home": [16, 24, 18, 8, 12, 13, 11, 10, 23, 28, 2]}

# Option 1: Using apply with a lambda function
viborg_players['Team'] = viborg_players['player_num'].apply(lambda x: 'home' if x in teams_dict["home"] else 'away')
viborg_players.loc[viborg_players['player_id'].str[:8].isin(away["away"]), 'Team'] = 'away'


In [ ]:
list(viborg_players['Team'][0:22])

In [ ]:
viborg_players.rename(columns={
    'player_id': 'unique_player_id',
    'player_num': 'id',
    'timestamp': 'time',
}, inplace=True)

In [ ]:
# Use only first period 

viborg_players = viborg_players[viborg_players["period"] == 1]
ball = ball[ball["period"] == 1]

In [ ]:
players_and_ball = viborg_players.merge(ball, on="time", how="left")

In [ ]:
players_and_ball = players_and_ball.dropna()
# Compute the Euclidean distance from the player (x, y) to the ball (ball_x, ball_y)
players_and_ball["distance_to_ball"] = np.sqrt((players_and_ball["x"] - players_and_ball["ball_x"])**2 +
                                          (players_and_ball["y"] - players_and_ball["ball_y"])**2)

# For each time point, rank the players by distance (1 = closest)
players_and_ball["distance_rank"] = players_and_ball.groupby("time")["distance_to_ball"].rank(method="min")
threshold = 3.0

# For each time point, count how many players are within the threshold distance to the ball.
# We use groupby with transform so that every row for the same time gets the same count.
players_and_ball["uncertainty_index"] = players_and_ball.groupby("time")["distance_to_ball"].transform(
    lambda x: (x <= threshold).sum()
)

rank_1= players_and_ball[players_and_ball["distance_rank"] == 1]
rank_1= players_and_ball[players_and_ball["distance_to_ball"] < 2]
rank_1_index= rank_1[rank_1["smoothed_acceleration_observed"] == 1]

In [ ]:
np.median(rank_1["distance_to_ball"])

In [ ]:
# Define a threshold for outliers (e.g., z-score > 3 or < -3)
import matplotlib.pyplot as plt

import seaborn as sns
# Create a density plot on the column 'my_column'
plt.figure(figsize=(10, 6))
sns.kdeplot(data=rank_1, x='distance_to_ball', fill=True, color='skyblue')
plt.title('Density Plot of my_column')
plt.xlabel('my_column')
plt.ylabel('Density')
plt.show()

In [ ]:
import numpy as np
import pandas as pd

def resolve_ties_by_team(df):
    """Resolve ties at the same timestamp by checking previous and next team's alignment."""
    unique_times = df["time"].unique()
    resolved = []
    for i, t in enumerate(unique_times):
        candidates = df[df["time"] == t]
        if len(candidates) == 1:
            resolved.append(candidates.iloc[0])
        else:
            # If we have a previous candidate, use its team.
            if resolved:
                prev_team = resolved[-1]["Team"]
            else:
                prev_team = None

            # Look at next unique time (if exists)
            if i < len(unique_times) - 1:
                next_time = unique_times[i+1]
                next_candidates = df[df["time"] == next_time]
                next_team = next_candidates.iloc[0]["Team"] if len(next_candidates) > 0 else None
            else:
                next_team = None

            chosen = None
            # 1) Try matching both prev_team & next_team.
            if prev_team and next_team:
                both = candidates[(candidates["Team"] == prev_team) & (candidates["Team"] == next_team)]
                if len(both) == 1:
                    chosen = both.iloc[0]
            # 2) If not, try matching prev_team.
            if chosen is None and prev_team:
                match_prev = candidates[candidates["Team"] == prev_team]
                if len(match_prev) == 1:
                    chosen = match_prev.iloc[0]
            # 3) If still not, try matching next_team.
            if chosen is None and next_team:
                match_next = candidates[candidates["Team"] == next_team]
                if len(match_next) == 1:
                    chosen = match_next.iloc[0]
            # 4) Fallback: choose the first candidate.
            if chosen is None:
                chosen = candidates.iloc[0]
            resolved.append(chosen)
    return pd.DataFrame(resolved).reset_index(drop=True)

def compress_consecutive_id(df):
    """
    Group consecutive rows with the same id into a single block with start/end times.
    Only blocks with at least 3 observations (count >= 3) are retained.
    """
    blocks = []
    current_block = None
    for _, row in df.iterrows():
        if current_block is None:
            # Start a new block with count 1.
            current_block = {
                "id": row["id"],
                "Team": row["Team"],
                "start_time": row["time"],
                "end_time": row["time"],
                "count": 1
            }
        else:
            if row["id"] == current_block["id"]:
                current_block["end_time"] = row["time"]
                current_block["count"] += 1
            else:
                # Only add the block if it has at least 3 observations.
                if current_block["count"] >= 3:
                    blocks.append(current_block)
                # Start a new block for the new id.
                current_block = {
                    "id": row["id"],
                    "Team": row["Team"],
                    "start_time": row["time"],
                    "end_time": row["time"],
                    "count": 1
                }
    if current_block and current_block["count"] >= 3:
        blocks.append(current_block)
    return pd.DataFrame(blocks)

def build_pass_events(blocks_df, rank_df, uncertainty_col="uncertainty_index"):
    """
    Create a pass event for each adjacent pair of blocks, but only if both blocks belong to the same team.
    
    For each pass event, defined as the transition between adjacent blocks in blocks_df,
    we compute the mean uncertainty over the time interval from the start time of the current block
    to the start time of the next block using values from rank_df.
    
    Returns a DataFrame with columns:
      - "Start Time [s]"
      - "End Time [s]"
      - "From"
      - "To"
      - "uncertainty"
      - "Team" (the team for the event)
    """
    blocks_df = blocks_df.sort_values("start_time").reset_index(drop=True)
    events = []
    for i in range(len(blocks_df) - 1):
        # Only create a pass event if both blocks are on the same team.
        if blocks_df.loc[i, "Team"] != blocks_df.loc[i+1, "Team"]:
            continue
        start_time = blocks_df.loc[i, "start_time"]
        end_time = blocks_df.loc[i+1, "start_time"]
        # Filter rows from rank_df with times between start_time and end_time.
        subset = rank_df[(rank_df["time"] >= start_time) & (rank_df["time"] <= end_time)]
        uncertainty_value = subset[uncertainty_col].mean() if not subset.empty else np.nan
        events.append({
            "Start Time [s]": start_time,
            "End Time [s]": end_time,
            "From": blocks_df.loc[i, "id"],
            "To": blocks_df.loc[i+1, "id"],
            "uncertainty": uncertainty_value,
            "Team": blocks_df.loc[i, "Team"]
        })
    return pd.DataFrame(events)

# -------------------------------
# Example usage:
# Assume 'rank_1_index' is your original DataFrame with at least columns "time", "id", "Team", and "uncertainty_index".

# 1) Resolve ties in your DataFrame.
df_resolved = resolve_ties_by_team(rank_1_index)

# 2) Compress consecutive rows by id but only keep blocks with at least 3 observations.
df_blocks = compress_consecutive_id(df_resolved)

# 3) Build pass events from the valid blocks, computing uncertainty over the interval.
#    Only events where both blocks belong to the same team are kept.
df_passes = build_pass_events(df_blocks, rank_1_index, uncertainty_col="uncertainty_index")

df_passes


In [ ]:
len(df_passes["To"].unique())

In [ ]:
# Filter rows where (End Time [s] - Start Time [s]) <= 7
df_filtered = df_passes[(df_passes["End Time [s]"] - df_passes["Start Time [s]"]) <= 10]

In [ ]:
len(df_filtered)

In [ ]:
df_filtered

In [ ]:
df_filtered.to_csv("test_for_streamit.csv")

In [ ]:
players_and_ball.to_csv("gps_video_for_streamlit.csv")

In [ ]:
df_filtered[df_filtered["From"] == 2]

In [ ]:
import io
import imageio
import matplotlib.pyplot as plt
import numpy as np

def create_video_of_events(players_and_ball, start_time, end_time, video_filename="events_video.mp4", fps=2):
    """
    Creates a video from player positions between start_time and end_time.
    Each frame shows the pitch and player positions (with their ids annotated).
    
    The pitch is drawn as a rectangle with x limits from 0 to 106 and y limits from 0 to 68,
    with the pitch center at (53,34).
    
    Parameters:
        players_and_ball (pd.DataFrame): DataFrame containing player positions. Expected columns: 
                                          "time", "x", "y", "id", "Team", and optionally "ball_x", "ball_y".
        start_time (float): The starting time for the video clip.
        end_time (float): The ending time for the video clip.
        video_filename (str): Output video file name.
        fps (int): Frames per second.
    """
    # Get all unique time points within the desired interval.
    times = sorted(players_and_ball["time"].unique())
    times = [t for t in times if start_time <= t <= end_time]
    
    frames = []
    
    # Define team colors (adjust colors as desired)
    unique_teams = players_and_ball["Team"].unique()
    team_colors = {team: color for team, color in zip(unique_teams, ["cornflowerblue", "tomato"])}
    
    for t in times:
        # Create a new figure and axis.
        fig, ax = plt.subplots(figsize=(8, 6))
        
        # Draw the pitch outline (0,0) to (106,68)
        pitch_outline = plt.Rectangle((0, 0), 106, 68, edgecolor="black", facecolor="none", lw=2)
        ax.add_patch(pitch_outline)
        
        # Draw the midline at x=53
        ax.axvline(x=53, color="black", lw=2)
        
        # Draw the center circle (approximate radius 9.15)
        center_circle = plt.Circle((53, 34), 9.15, color="black", fill=False, lw=2)
        ax.add_patch(center_circle)
        
        # Draw the center spot
        ax.scatter(53, 34, color="black", s=30)
        
        # Set the axis limits to the pitch dimensions
        ax.set_xlim(0, 106)
        ax.set_ylim(0, 68)
        
        # Filter the DataFrame for the current time.
        current_positions = players_and_ball[players_and_ball["time"] == t].copy()
        current_positions["color"] = current_positions["Team"].map(team_colors)
        
        # Plot player positions.
        ax.scatter(current_positions["x"], current_positions["y"],
                   edgecolor="black", facecolor=current_positions["color"], alpha=0.8)
        
        # Plot the ball if ball_x and ball_y exist.
        if "ball_x" in current_positions.columns and "ball_y" in current_positions.columns:
            ax.scatter(current_positions["ball_x"], current_positions["ball_y"],
                       edgecolor="black", facecolor="black", alpha=1, s=50)
        
        # Annotate each player with their id.
        for _, row in current_positions.iterrows():
            ax.text(row["x"], row["y"], str(row["id"]),
                    color="black", weight="bold", ha="center", va="center")
        
        ax.set_title(f"Time: {t:.2f} s")
        
        # Save the figure to a BytesIO buffer.
        buf = io.BytesIO()
        fig.savefig(buf, format="png", bbox_inches="tight")
        buf.seek(0)
        image = imageio.imread(buf)
        frames.append(image)
        plt.close(fig)
    
    # Write the frames to a video file.
    imageio.mimwrite(video_filename, frames, fps=fps)



In [ ]:
import io
import imageio
import matplotlib.pyplot as plt
import numpy as np

def create_video_of_events(players_and_ball, start_time, end_time, video_filename="events_video.mp4", fps=2, highlight_ids=None):
    """
    Creates a video from player positions between start_time and end_time.
    Each frame shows the pitch and player positions. Only the players whose IDs are in 
    'highlight_ids' are annotated and are plotted in a different color.
    
    The pitch is drawn as a rectangle with x limits from 0 to 106 and y limits from 0 to 68,
    with the pitch center at (53,34).
    
    Parameters:
        players_and_ball (pd.DataFrame): DataFrame containing player positions.
            Expected columns: "time", "x", "y", "id", "Team", and optionally "ball_x", "ball_y".
        start_time (float): The starting time for the video clip.
        end_time (float): The ending time for the video clip.
        video_filename (str): Output video file name.
        fps (int): Frames per second.
        highlight_ids (list): List of player IDs (matching the "id" column) that should be highlighted.
    """
    if highlight_ids is None:
        highlight_ids = []
    
    # Get all unique time points within the desired interval.
    times = sorted(players_and_ball["time"].unique())
    times = [t for t in times if start_time <= t <= end_time]
    
    frames = []
    
    # Define team colors (adjust colors as desired)
    unique_teams = players_and_ball["Team"].unique()
    team_colors = {team: color for team, color in zip(unique_teams, ["cornflowerblue", "tomato"])}
    
    for t in times:
        # Create a new figure and axis.
        fig, ax = plt.subplots(figsize=(8, 6))
        
        # Draw the pitch outline (0,0) to (106,68)
        pitch_outline = plt.Rectangle((0, 0), 106, 68, edgecolor="black", facecolor="none", lw=2)
        ax.add_patch(pitch_outline)
        
        # Draw the midline at x=53
        ax.axvline(x=53, color="black", lw=2)
        
        # Draw the center circle (approximate radius 9.15)
        center_circle = plt.Circle((53, 34), 9.15, color="black", fill=False, lw=2)
        ax.add_patch(center_circle)
        
        # Draw the center spot
        ax.scatter(53, 34, color="black", s=30)
        
        # Set the axis limits to the pitch dimensions
        ax.set_xlim(0, 106)
        ax.set_ylim(0, 68)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        
        # Filter the DataFrame for the current time.
        current_positions = players_and_ball[players_and_ball["time"] == t].copy()
        current_positions["color"] = current_positions["Team"].map(team_colors)
        
        # Separate positions into highlighted and non-highlighted.
        highlighted = current_positions[current_positions["id"].isin(highlight_ids)]
        others = current_positions[~current_positions["id"].isin(highlight_ids)]
        
        # Plot non-highlighted player positions.
        ax.scatter(others["x"], others["y"],
                   edgecolor="black", facecolor=others["color"], alpha=0.8)
        
        # Plot highlighted player positions with a different color and larger markers.
        ax.scatter(highlighted["x"], highlighted["y"],
                   edgecolor="black", facecolor="gold", alpha=0.9, s=100, zorder=5)
        
        # Annotate only the highlighted players.
        for _, row in highlighted.iterrows():
            ax.text(row["x"], row["y"], str(row["id"]),
                    color="black", weight="bold", ha="center", va="center", fontsize=12)
        
        # Plot the ball if ball_x and ball_y exist.
        if "ball_x" in current_positions.columns and "ball_y" in current_positions.columns:
            ax.scatter(current_positions["ball_x"], current_positions["ball_y"],
                       edgecolor="black", facecolor="black", alpha=1, s=50)
        
        ax.set_title(f"Time: {t:.2f} s")
        
        # Save the figure to a BytesIO buffer.
        buf = io.BytesIO()
        fig.savefig(buf, format="png", bbox_inches="tight")
        buf.seek(0)
        image = imageio.imread(buf)
        frames.append(image)
        plt.close(fig)
    
    # Write the frames to a video file.
    imageio.mimwrite(video_filename, frames, fps=fps)


In [ ]:
create_video_of_events(players_and_ball,748.60,	751.40, video_filename="events_video.mp4", fps=10) #highlight_ids=[2,11])

In [ ]:
aligned_groups, sorted_df   = find_vertical_aligned_groups(viborg_players[0:100])

In [ ]:
viborg_players

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon
from io import StringIO

# Define the formation data as CSV strings.
data_442 = """formation,role,player,position_x,position_y
4-4-2,Goalkeeper,1,5,34
4-4-2,Defender,2,20,10
4-4-2,Defender,3,20,25
4-4-2,Defender,4,20,43
4-4-2,Defender,5,20,58
4-4-2,Midfielder,6,50,10
4-4-2,Midfielder,7,50,25
4-4-2,Midfielder,8,50,43
4-4-2,Midfielder,9,50,58
4-4-2,Forward,10,80,25
4-4-2,Forward,11,80,43
"""

data_433 = """formation,role,player,position_x,position_y
4-3-3,Goalkeeper,1,5,34
4-3-3,Defender,2,20,15
4-3-3,Defender,3,20,30
4-3-3,Defender,4,20,38
4-3-3,Defender,5,20,53
4-3-3,Midfielder,6,45,20
4-3-3,Midfielder,7,45,34
4-3-3,Midfielder,8,45,48
4-3-3,Forward,9,70,10
4-3-3,Forward,10,75,34
4-3-3,Forward,11,70,58
"""

data_4231 = """formation,role,player,position_x,position_y
4-2-3-1,Goalkeeper,1,5,34
4-2-3-1,Defender,2,20,15
4-2-3-1,Defender,3,20,28
4-2-3-1,Defender,4,20,40
4-2-3-1,Defender,5,20,53
4-2-3-1,Defensive Midfielder,6,35,25
4-2-3-1,Defensive Midfielder,7,35,43
4-2-3-1,Attacking Midfielder,8,55,15
4-2-3-1,Attacking Midfielder,9,60,34
4-2-3-1,Attacking Midfielder,10,55,53
4-2-3-1,Forward,11,80,34
"""

# --- Create DataFrames from CSV strings ---
df_442 = pd.read_csv(StringIO(data_442))
df_433 = pd.read_csv(StringIO(data_433))
df_4231 = pd.read_csv(StringIO(data_4231))

# --- Compute Convex Hull ---
def compute_convex_hull(df):
    """
    Computes the convex hull for the player positions.
    
    Returns:
      hull: The ConvexHull object.
      hull_points: An array of (x, y) vertices (in order) of the convex hull.
    """
    points = df[['position_x', 'position_y']].values
    if len(points) < 3:
        return None, points
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    return hull, hull_points

# --- Group Players by Vertical Columns ---
def group_vertical_segments(df, tol=1.0):
    """
    Groups players by similar x coordinates (using a tolerance) to form vertical segments.
    
    Returns:
      segments: List of tuples (x_group, min_y, max_y) for each vertical group.
    """
    # Create a grouping key by rounding the x coordinate to the nearest 'tol'
    df = df.copy()
    df['x_group'] = (df['position_x'] / tol).round() * tol
    segments = []
    for group_val, group in df.groupby('x_group'):
        min_y = group['position_y'].min()
        max_y = group['position_y'].max()
        segments.append((group_val, min_y, max_y))
    return segments

# --- Plotting Function ---
def plot_formation(ax, df, formation_name, tol=1.0):
    """
    Plots the player positions, overlays the convex hull, and draws vertical connections.
    
    Parameters:
      ax: Matplotlib axis object.
      df: DataFrame with columns 'position_x' and 'position_y'.
      formation_name: String with the formation name.
      tol: Tolerance used for grouping vertical segments.
    """
    # Plot all player positions
    ax.scatter(df['position_x'], df['position_y'], color='blue', zorder=3, label='Players')
    
    # Compute and plot the convex hull
    hull, hull_points = compute_convex_hull(df)
    if hull_points is not None and len(hull_points) > 0:
        # Ensure the polygon is closed by appending the first point at the end
        hull_polygon = np.vstack([hull_points, hull_points[0]])
        ax.plot(hull_polygon[:,0], hull_polygon[:,1], 'r--', lw=2, label='Convex Hull')
        num_hull_edges = len(hull_points)
    else:
        num_hull_edges = 0
    
    # Group players by vertical columns and plot vertical segments
    vertical_segments = group_vertical_segments(df, tol=tol)
    for i, seg in enumerate(vertical_segments):
        x, min_y, max_y = seg
        ax.plot([x, x], [min_y, max_y], 'g-', lw=2,
                label='Vertical Segments' if i == 0 else "")
        # Optionally mark the endpoints
        ax.plot(x, min_y, 'ko', markersize=4)
        ax.plot(x, max_y, 'ko', markersize=4)
    
    # Annotate the plot with details
    ax.text(0.5, 0.95, f"Hull Edges: {num_hull_edges}\nVertical Segments: {len(vertical_segments)}",
            transform=ax.transAxes, fontsize=10, ha="center", va="top",
            bbox=dict(facecolor="white", alpha=0.7))
    
    # Set pitch boundaries (assuming a standard pitch 105 x 68 m)
    ax.set_xlim(0, 105)
    ax.set_ylim(0, 68)
    ax.set_title(formation_name)
    ax.set_xlabel("Pitch Length (m)")
    ax.set_ylabel("Pitch Width (m)")
    ax.set_aspect("equal")
    ax.grid(True, linestyle="--", alpha=0.5)
    ax.legend()

# --- Main Script ---
if __name__ == "__main__":
    # Create a figure with three subplots (one per formation)
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    
    formations = [("Formation 4-4-2", df_442),
                  ("Formation 4-3-3", df_433),
                  ("Formation 4-2-3-1", df_4231)]
    
    for ax, (formation_name, df) in zip(axs, formations):
        plot_formation(ax, df, formation_name, tol=1.0)
    
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from io import StringIO

# --- Formation Data as CSV strings ---
data_442 = """formation,role,player,position_x,position_y
4-4-2,Goalkeeper,1,5,34
4-4-2,Defender,2,20,10
4-4-2,Defender,3,20,25
4-4-2,Defender,4,20,43
4-4-2,Defender,5,20,58
4-4-2,Midfielder,6,50,10
4-4-2,Midfielder,7,50,25
4-4-2,Midfielder,8,50,43
4-4-2,Midfielder,9,50,58
4-4-2,Forward,10,80,25
4-4-2,Forward,11,80,43
"""

data_433 = """formation,role,player,position_x,position_y
4-3-3,Goalkeeper,1,5,34
4-3-3,Defender,2,20,15
4-3-3,Defender,3,20,30
4-3-3,Defender,4,20,38
4-3-3,Defender,5,20,53
4-3-3,Midfielder,6,45,20
4-3-3,Midfielder,7,45,34
4-3-3,Midfielder,8,45,48
4-3-3,Forward,9,70,10
4-3-3,Forward,10,75,34
4-3-3,Forward,11,70,58
"""

data_4231 = """formation,role,player,position_x,position_y
4-2-3-1,Goalkeeper,1,5,34
4-2-3-1,Defender,2,20,15
4-2-3-1,Defender,3,20,28
4-2-3-1,Defender,4,20,40
4-2-3-1,Defender,5,20,53
4-2-3-1,Defensive Midfielder,6,35,25
4-2-3-1,Defensive Midfielder,7,35,43
4-2-3-1,Attacking Midfielder,8,55,15
4-2-3-1,Attacking Midfielder,9,60,34
4-2-3-1,Attacking Midfielder,10,55,53
4-2-3-1,Forward,11,80,34
"""

# --- Create DataFrames ---
df_442 = pd.read_csv(StringIO(data_442))
df_433 = pd.read_csv(StringIO(data_433))
df_4231 = pd.read_csv(StringIO(data_4231))

# --- Function to compute the convex hull of player positions ---
def compute_convex_hull(df):
    points = df[['position_x', 'position_y']].values
    if len(points) < 3:
        return None, points
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    return hull, hull_points

# --- Function to group players into vertical segments ---
def group_vertical_segments(df, tol=1.0):
    """
    Groups players by similar x coordinates (using a tolerance) to form vertical segments.
    Returns a list of tuples: (x_group, min_y, max_y)
    """
    df = df.copy()
    df['x_group'] = (df['position_x'] / tol).round() * tol
    segments = []
    for group_val, group in df.groupby('x_group'):
        min_y = group['position_y'].min()
        max_y = group['position_y'].max()
        segments.append((group_val, min_y, max_y))
    return segments

# --- Compute a polygon "fingerprint" (descriptor) ---
def get_polygon_features(df, tol=1.0):
    """
    Computes features from the formation:
      - Convex hull area
      - Convex hull perimeter
      - Number of hull edges
      - Number of vertical segments
      - Total vertical segment length
    """
    points = df[['position_x', 'position_y']].values
    if len(points) >= 3:
        hull, hull_points = compute_convex_hull(df)
        hull_area = hull.volume  # For 2D, volume is area.
        # Close the hull polygon to compute perimeter
        hull_closed = np.vstack([hull_points, hull_points[0]])
        hull_perimeter = np.sum(np.sqrt(np.sum(np.diff(hull_closed, axis=0)**2, axis=1)))
        num_hull_edges = len(hull_points)
    else:
        hull_area = 0
        hull_perimeter = 0
        num_hull_edges = 0

    segments = group_vertical_segments(df, tol)
    num_vertical_segments = len(segments)
    total_vertical_length = sum(max_y - min_y for (_, min_y, max_y) in segments)

    descriptor = {
        'hull_area': hull_area,
        'hull_perimeter': hull_perimeter,
        'num_hull_edges': num_hull_edges,
        'num_vertical_segments': num_vertical_segments,
        'total_vertical_length': total_vertical_length
    }
    return descriptor

# --- Function to compute a simple distance between two descriptors ---
def descriptor_distance(desc1, desc2, weights=None):
    """
    Computes a weighted distance between two polygon descriptors.
    You can adjust the weights as needed.
    """
    # Default weights (tuned for typical pitch dimensions)
    if weights is None:
        weights = {
            'hull_area': 1/1000,            # area in m^2 (normalized)
            'hull_perimeter': 1/100,        # perimeter in m
            'num_hull_edges': 5.0,          # discrete penalty if different
            'num_vertical_segments': 2.0,   # discrete penalty if different
            'total_vertical_length': 1/100  # in m
        }
    d = 0.0
    d += weights['hull_area'] * abs(desc1['hull_area'] - desc2['hull_area'])
    d += weights['hull_perimeter'] * abs(desc1['hull_perimeter'] - desc2['hull_perimeter'])
    # For discrete values, add penalty if not equal
    d += weights['num_hull_edges'] * abs(desc1['num_hull_edges'] - desc2['num_hull_edges'])
    d += weights['num_vertical_segments'] * abs(desc1['num_vertical_segments'] - desc2['num_vertical_segments'])
    d += weights['total_vertical_length'] * abs(desc1['total_vertical_length'] - desc2['total_vertical_length'])
    return d

# --- Search function ---
def search_polygon(query_desc, database):
    """
    Given a query polygon descriptor and a database (dict of {name: descriptor}),
    returns the name and descriptor of the best match.
    """
    best_name = None
    best_desc = None
    best_dist = float('inf')
    for name, desc in database.items():
        d = descriptor_distance(query_desc, desc)
        if d < best_dist:
            best_dist = d
            best_name = name
            best_desc = desc
    return best_name, best_desc, best_dist

# --- Plotting function to visualize formation features ---
def plot_formation(ax, df, formation_name, tol=1.0):
    # Plot player positions
    ax.scatter(df['position_x'], df['position_y'], color='blue', zorder=3, label='Players')
    
    # Plot convex hull if available
    hull, hull_points = compute_convex_hull(df)
    if hull_points is not None and len(hull_points) > 0:
        hull_closed = np.vstack([hull_points, hull_points[0]])
        ax.plot(hull_closed[:,0], hull_closed[:,1], 'r--', lw=2, label='Convex Hull')
    
    # Plot vertical segments
    segments = group_vertical_segments(df, tol)
    for i, (x, min_y, max_y) in enumerate(segments):
        ax.plot([x, x], [min_y, max_y], 'g-', lw=2,
                label='Vertical Segment' if i == 0 else "")
        ax.plot(x, min_y, 'ko', markersize=4)
        ax.plot(x, max_y, 'ko', markersize=4)
    
    ax.set_xlim(0, 105)
    ax.set_ylim(0, 68)
    ax.set_title(formation_name)
    ax.set_xlabel("Pitch Length (m)")
    ax.set_ylabel("Pitch Width (m)")
    ax.set_aspect("equal")
    ax.grid(True, linestyle="--", alpha=0.5)
    ax.legend()

# --- Main Script ---
if __name__ == "__main__":
    # Build a database of formation descriptors
    formations = {
        "4-4-2": df_442,
        "4-3-3": df_433,
        "4-2-3-1": df_4231
    }
    
    database = {}
    for name, df in formations.items():
        desc = get_polygon_features(df, tol=1.0)
        database[name] = desc
        print(f"Descriptor for {name}:")
        for k, v in desc.items():
            print(f"  {k}: {v}")
        print("-" * 40)
    
    # Simulate a query:
    # For example, we take the 4-3-3 formation and warp it slightly (simulate stretching)
    df_query = df_433.copy()
    # Apply a slight horizontal stretch (e.g., multiply x by 1.05) and vertical shift
    df_query['position_x'] = df_query['position_x'] * 1.05
    df_query['position_y'] = df_query['position_y'] + 0.5
    
    query_desc = get_polygon_features(df_query, tol=1.0)
    print("Query Descriptor:")
    for k, v in query_desc.items():
        print(f"  {k}: {v}")
    
    # Search for the best match in our database
    best_name, best_desc, best_dist = search_polygon(query_desc, database)
    print(f"\nBest match for query: {best_name} (distance = {best_dist:.3f})")
    
    # Plot the query formation and the matched formation side by side
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))
    plot_formation(axs[0], df_query, f"Query (Warped 4-3-3)", tol=1.0)
    plot_formation(axs[1], formations[best_name], f"Matched Formation: {best_name}", tol=1.0)
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from io import StringIO

def compute_convex_hull(df):
    """
    Computes the convex hull for the player positions (columns 'x' and 'y').
    Returns the ConvexHull object and an array of hull vertices.
    """
    points = df[['x', 'y']].values
    if len(points) < 3:
        return None, points
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    return hull, hull_points

def group_vertical_segments(df, tol=1.0):
    """
    Groups players by similar x coordinates using a tolerance.
    Returns a list of segments as tuples: (x_group, min_y, max_y).
    """
    df = df.copy()
    df['x_group'] = (df['x'] / tol).round() * tol
    segments = []
    for group_val, group in df.groupby('x_group'):
        min_y = group['y'].min()
        max_y = group['y'].max()
        segments.append((group_val, min_y, max_y))
    return segments

def plot_formation_for_time(df, time_val, tol=1.0):
    """
    Given the DataFrame df for a specific time, plots:
      - A scatter of player positions.
      - The convex hull polygon.
      - Vertical segments connecting players close in x.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot player positions
    ax.scatter(df['x'], df['y'], color='blue', zorder=3, label='Players')
    
    # Compute and plot convex hull
    hull, hull_points = compute_convex_hull(df)
    if hull_points is not None and len(hull_points) > 0:
        hull_closed = np.vstack([hull_points, hull_points[0]])
        ax.plot(hull_closed[:, 0], hull_closed[:, 1], 'r--', lw=2, label='Convex Hull')
    
    # Group by x coordinate and plot vertical segments
    segments = group_vertical_segments(df, tol)
    for i, (x_group, min_y, max_y) in enumerate(segments):
        ax.plot([x_group, x_group], [min_y, max_y], 'g-', lw=2,
                label='Vertical Segment' if i == 0 else "")
        ax.plot(x_group, min_y, 'ko', markersize=4)
        ax.plot(x_group, max_y, 'ko', markersize=4)
    
    # Set pitch boundaries (adjust as needed)
    ax.set_xlim(0, 105)
    ax.set_ylim(0, 68)
    ax.set_title(f"Team Formation at time = {time_val:.2f}s")
    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")
    ax.set_aspect('equal')
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.show()

def main():
    # --- Load Your Data ---
    # Replace this with your own file read, e.g., pd.read_csv("your_data.csv", sep="\t")

    # --- Filter to "home" team ---
    df_home = viborg_players[viborg_players['Team'] == 'home'].copy()
    df_home['time'] = df_home['time'].astype(float)
    
    # --- Select Time Frames ---
    # For every 5th second, we assume the 'time' column holds seconds.
    unique_times = sorted(df_home['time'].unique())
    # Only select times that are multiples of 5 (allowing for floating point precision)
    times_to_plot = [t for t in unique_times if abs(t - round(t / 5) * 5) < 1e-3]
    
    # For each selected time, filter the data and plot the formation polygon
    for t in times_to_plot:
        df_time = df_home[df_home['time'] == t]
        if len(df_time) < 3:
            print(f"Time {t} has less than 3 players, skipping.")
            continue
        plot_formation_for_time(df_time, t, tol=3.0)

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from io import StringIO

def compute_convex_hull(df):
    """
    Computes the convex hull for the player positions (columns 'x' and 'y').
    Returns the ConvexHull object and an array of hull vertices.
    """
    points = df[['x', 'y']].values
    if len(points) < 3:
        return None, points
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    return hull, hull_points

def group_players_by_x(df, tol=1.0):
    """
    Groups players by rounding their x-coordinate to the nearest multiple of `tol`.
    Players whose x-coordinates round to the same value are considered in the same group.
    """
    df = df.copy()
    # Rounding to the nearest multiple of tol
    df['x_group'] = (df['x'] / tol).round() * tol
    return df

def plot_formation_for_time(df, time_val, tol=1.0):
    """
    For a specific time slice (df contains players at that time), plots:
      - Player positions (blue dots)
      - Convex hull of the positions (red dashed line)
      - Line segments connecting consecutive players in each x_group 
        (sorted by y) using their actual (x,y) coordinates.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot player positions
    ax.scatter(df['x'], df['y'], color='blue', zorder=3, label='Players')
    
    # Compute and plot the convex hull
    hull, hull_points = compute_convex_hull(df)
    if hull_points is not None and len(hull_points) > 0:
        hull_closed = np.vstack([hull_points, hull_points[0]])
        ax.plot(hull_closed[:, 0], hull_closed[:, 1], 'r--', lw=2, label='Convex Hull')
    
    # Group players by x (within tolerance) and connect consecutive players in ascending y
    df_grouped = group_players_by_x(df, tol=tol)
    for xg_val, group in df_grouped.groupby('x_group'):
        # Sort each group by y
        group_sorted = group.sort_values('y')
        # Connect consecutive players in this group
        for i in range(len(group_sorted) - 1):
            x1, y1 = group_sorted.iloc[i][['x', 'y']]
            x2, y2 = group_sorted.iloc[i+1][['x', 'y']]
            # Draw a line between these actual coordinates (may be angled if x1 != x2)
            ax.plot([x1, x2], [y1, y2], 'g-', lw=2,
                    label='Within X-Tolerance' if i == 0 else "")
            # Optionally mark the endpoints
            ax.plot(x1, y1, 'ko', markersize=4)
            ax.plot(x2, y2, 'ko', markersize=4)
    
    # Set pitch boundaries (adjust as needed, here assumed 105x68)
    ax.set_xlim(0, 105)
    ax.set_ylim(0, 68)
    ax.set_title(f"Team Formation at time = {time_val:.2f}s")
    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")
    ax.set_aspect('equal')
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.show()

def main():

    df_home = viborg_players[viborg_players['Team'] == 'home'].copy()
    df_home['time'] = df_home['time'].astype(float)
    # --- Select every 5th-second time slice ---
    # (In this example, only t=0 is present, but you'd do something like:)
    unique_times = sorted(df_home['time'].unique())
    times_to_plot = [t for t in unique_times if abs(t - round(t / 5) * 5) < 1e-3]
    
    # For each selected time, filter and plot the formation
    for t in times_to_plot:
        df_time = df_home[df_home['time'] == t]
        if len(df_time) < 3:
            print(f"Time {t} has less than 3 players, skipping.")
            continue
        plot_formation_for_time(df_time, t, tol=10.0)

if __name__ == "__main__":
    main()
